In [1]:
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels
import hvplot.xarray

In [2]:
station = "Hot"

## Load DATA


### CMEMS


In [3]:
data_cmems = xr.open_dataset(f"../data/1_products/{station}_cmems.zarr", engine="zarr").load()
data_cmems

<xarray.Dataset> Size: 2MB
Dimensions:              (latitude: 1, longitude: 1, time: 9131, depth: 3)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * time                 (time) datetime64[ns] 73kB 1998-01-01 ... 2022-12-31
Data variables: (12/13)
    T                    (latitude, longitude, time, depth) float64 219kB 23....
    U                    (latitude, longitude, time, depth) float64 219kB 0.0...
    V                    (latitude, longitude, time, depth) float64 219kB -0....
    mnkc_epi             (latitude, longitude, time) float64 73kB 0.2672 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 73kB 0.7941 ... ...
    mnkc_lmeso           (latitude, longitude, time) float64 73kB 2.175 ... 3...
    ...                   ...
    mnkc_mumeso          (latitude, longitude, time) float64 73kB 0.4954 ... ...
    mnkc_umeso           (latitude, longitude, time) float64 73kB 0.4923 ... ...
    npp                  (latitude, longitude, time) float64 73kB 315.8 ... 2...
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 219kB 117...
    zeu                  (latitude, longitude, time) float64 73kB 78.19 ... 8...
    zooc                 (latitude, longitude, time) float64 73kB 0.2889 ... ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

## Climato


In [4]:
for i in range(1, 13):
    print(
        f"Month {i} : ",
        set(
            pd.DataFrame({"month": data_cmems.indexes["time"].month, "day": data_cmems.indexes["time"].day})
            .query(f"month == {i}")["day"]
            .unique()
        ),
    )

Month 1 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 2 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29}
Month 3 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 4 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
Month 5 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 6 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
Month 7 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 8 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}


In [5]:
data_cmems_climato = data_cmems.groupby("time.dayofyear").mean()
data_cmems_climato

<xarray.Dataset> Size: 64kB
Dimensions:              (dayofyear: 366, latitude: 1, longitude: 1, depth: 3)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * dayofyear            (dayofyear) int64 3kB 1 2 3 4 5 ... 362 363 364 365 366
Data variables: (12/13)
    T                    (dayofyear, latitude, longitude, depth) float64 9kB ...
    U                    (dayofyear, latitude, longitude, depth) float64 9kB ...
    V                    (dayofyear, latitude, longitude, depth) float64 9kB ...
    mnkc_epi             (dayofyear, latitude, longitude) float64 3kB 0.2636 ...
    mnkc_hmlmeso         (dayofyear, latitude, longitude) float64 3kB 0.755 ....
    mnkc_lmeso           (dayofyear, latitude, longitude) float64 3kB 2.135 ....
    ...                   ...
    mnkc_mumeso          (dayofyear, latitude, longitude) float64 3kB 0.527 ....
    mnkc_umeso           (dayofyear, latitude, longitude) float64 3kB 0.4773 ...
    npp                  (dayofyear, latitude, longitude) float64 3kB 304.7 ....
    pelagic_layer_depth  (dayofyear, latitude, longitude, depth) float64 9kB ...
    zeu                  (dayofyear, latitude, longitude) float64 3kB 76.92 ....
    zooc                 (dayofyear, latitude, longitude) float64 3kB 0.2564 ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

## Expand to original time series interval


In [6]:
all_years = []
for year in range(int(data_cmems.time.min().dt.year), int(data_cmems.time.max().dt.year + 1)):
    slice_max = 366
    if not pd.Timestamp(f"{year}-12-31").is_leap_year:
        slice_max = 365

    yearly_data = data_cmems_climato.sel(dayofyear=slice(1, slice_max)).assign_coords(
        time=(
            "dayofyear",
            pd.date_range(
                start=f"{year}-01-01",
                end=f"{year}-12-31",
                freq="D",
                name="time",
            ),
        ),
    )

    all_years.append(yearly_data.swap_dims({"dayofyear": "time"}))
data_cmems_climato_expanded = xr.concat(all_years, dim="time").drop_vars("dayofyear")
data_cmems_climato_expanded.time.attrs = data_cmems.time.attrs
data_cmems_climato_expanded

<xarray.Dataset> Size: 2MB
Dimensions:              (time: 9131, latitude: 1, longitude: 1, depth: 3)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * time                 (time) datetime64[ns] 73kB 1998-01-01 ... 2022-12-31
Data variables: (12/13)
    T                    (time, latitude, longitude, depth) float64 219kB 23....
    U                    (time, latitude, longitude, depth) float64 219kB -0....
    V                    (time, latitude, longitude, depth) float64 219kB -0....
    mnkc_epi             (time, latitude, longitude) float64 73kB 0.2636 ... ...
    mnkc_hmlmeso         (time, latitude, longitude) float64 73kB 0.755 ... 0...
    mnkc_lmeso           (time, latitude, longitude) float64 73kB 2.135 ... 2...
    ...                   ...
    mnkc_mumeso          (time, latitude, longitude) float64 73kB 0.527 ... 0...
    mnkc_umeso           (time, latitude, longitude) float64 73kB 0.4773 ... ...
    npp                  (time, latitude, longitude) float64 73kB 304.7 ... 3...
    pelagic_layer_depth  (time, latitude, longitude, depth) float64 219kB 115...
    zeu                  (time, latitude, longitude) float64 73kB 76.92 ... 7...
    zooc                 (time, latitude, longitude) float64 73kB 0.2564 ... ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

## Plot


In [7]:
data_cmems_climato_expanded.mean(["latitude", "longitude"]).hvplot.line(
    x="time",
    groupby="depth",
    title=f"Climato {station} - CMEMS T",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'5829e452-4b24-4ea4-8c22-6b6f4cd12b39': {'version…

## Export


In [8]:
data_cmems_climato_expanded.to_zarr(
    f"../data/1_products/{station}_cmems_climato.zarr",
    mode="w",
)